### 잡플래닛 기업 리뷰 크롤링
- 크롤링 과정
    1. 회사 이름 검색
    2. 리뷰 link 얻기
    3. 리뷰 내용 얻기
        1. review_text : title,strength,weakness,want (로그인 필요 -> requests.session 사용)
        2. stats : request 다시
        3. review_num, person, company_name : request 다시

### scrapy

In [4]:
!rm -rf job_search
!scrapy startproject job_search

New Scrapy project 'job_search', using template directory '/Users/juhyunson/opt/anaconda3/lib/python3.7/site-packages/scrapy/templates/project', created in:
    /Users/juhyunson/Documents/dss0/Programming/crawling 프로젝트/job_search

You can start your first spider with:
    cd job_search
    scrapy genspider example example.com


In [5]:
!cat job_search/job_search/items.py

# -*- coding: utf-8 -*-

# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class JobSearchItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    pass


In [6]:
%%writefile job_search/job_search/items.py
import scrapy


class JobSearchItem(scrapy.Item):
    link=scrapy.Field()
    company_id=scrapy.Field()
    company_name=scrapy.Field()
    title = scrapy.Field()
    strength = scrapy.Field()
    weakness = scrapy.Field()
    want=scrapy.Field()
    stats = scrapy.Field()    
    review_num=scrapy.Field()
    person=scrapy.Field()

Overwriting job_search/job_search/items.py


In [2]:
%%writefile job_search/job_search/spiders/spider.py
import scrapy
import requests
from bs4 import BeautifulSoup
from scrapy.http import TextResponse
from job_search.items import JobSearchItem
import configparser


class Spider(scrapy.Spider):
    name="JobSearch"
    
    def __init__(self,company_name='구글코리아',**kwargs):
        self.company_name=company_name
        self.start_urls='https://www.jobplanet.co.kr/search?query={}&category=&_rs_con=welcome&_rs_act=index&_rs_element=main_search_bar'.format(company_name)
        super().__init__(**kwargs)
        
    def start_requests(self):
        url=self.start_urls
        yield scrapy.Request(url,callback=self.parse)
        
    def parse(self,response):
        item=JobSearchItem()
        #로그인 데이터 불러오기
        config = configparser.ConfigParser()
        config.read('login.ini')
        link_first=response.xpath('//*[@id="mainContents"]/div[1]/div/div[2]/div[1]/div/a/@href').extract()[0]
        link_first=response.urljoin(link_first)
        item['link']=link_first
        
        #모든 page의 reveiw_text,person 크롤링
        page=1
        titles_ls,strength_ls,weakness_ls,wants_ls,person_ls=[],[],[],[],[]
        while True:
            link=link_first.split('info/')[0]+'reviews/'+self.company_name+'?page={}'.format(page)
        
            #link 접속 
            login_url = 'https://www.jobplanet.co.kr/users/sign_in'
            login_data = {'user': {'email': config.get('section1', 'ID'), 'password': config.get('section1', 'PW'), 'remember_me':'true'}}
            session = requests.session()
            req = session.post(login_url, json = login_data) 
            req = session.get(link)
            response_s=TextResponse(req.url, body=req.text, encoding='utf-8')
            
            #review_text
            elements=response_s.xpath('//*[@id="viewReviewsList"]/div/div/div/section/div/div[2]/div/div[1]/h2/text()').extract()
            titles = [element.replace("\n", "").strip() for element in elements]
            titles = " ".join(titles)
            #title의 길이가 0일때까지 page의 수를 증가시킨다.
            if len(titles)==0: break
            titles_ls.append(titles)
            
            elements=response_s.xpath('//*[@id="viewReviewsList"]/div/div/div/section/div/div[2]/div/dl/dd[1]/span/text()').extract()
            strengths = [element.replace("\n", "").replace("\r","").strip() for element in elements]
            strengths = " | ".join(strengths)
            strength_ls.append(strengths)
            
            elements=response_s.xpath('//*[@id="viewReviewsList"]/div/div/div/section/div/div[2]/div/dl/dd[2]/span/text()').extract()
            weakness = [element.replace("\n", "").replace("\r","").strip() for element in elements]
            weakness = " | ".join(weakness)
            weakness_ls.append(weakness)
            
            elements=response_s.xpath('//*[@id="viewReviewsList"]/div/div/div/section/div/div[2]/div/dl/dd[3]/span/text()').extract()
            wants = [element.replace("\n", "").replace("\r","").strip() for element in elements]
            wants = " | ".join(wants)
            wants_ls.append(wants)
            
            #person
            person=response_s.xpath('//*[@id="viewReviewsList"]/div/div/div/section/div/div[1]/span/text()').extract()
            def divide_list(l, n=7): 
                # 리스트 l의 길이가 n이면 계속 반복
                for i in range(0, len(l), n): 
                    yield l[i:i + n] 
            person=list(divide_list(person))[0][::2]
            person_ls.append(person)
            
            page+=1
            
        item['title']=titles_ls
        item['strength']=strength_ls
        item['weakness']=weakness_ls
        item['want']=wants_ls
        item['person'] = person_ls
        item['company_name']=self.company_name
        company_id = link_first.split("companies/")[1].split('/info')[0]
        item['company_id']=company_id

        #stats 
        #request 다시(json으로)
        def stats(company_id=company_id):
            url = "https://www.jobplanet.co.kr/api/v3/companies/reviews/premium_rating?company_id={}".format(company_id)
            response2 = requests.get(url)
            contents = response2.json()["data"]["contents"]
            items = contents[0]["graph_data"]["bar_data"]["items"]
            stats = {}
            for item in items:
                stats[item["label"]] = item["val"]
            return stats
        
        item['stats']=stats(company_id)
        
        #review_num
        #request 다시(xpath으로. item을 같은 행에 return 하기 위하여 같은 함수에 request를 적용해줌)
        url = link_first
        req = requests.get(url)
        response3=TextResponse(req.url, body=req.text, encoding='utf-8')
        item['review_num']=response3.xpath('//*[@id="viewCompaniesMenu"]/ul/li[2]/a/span/text()').extract()[0].replace("\n","").replace(" ","")
       
        

        
        
        yield item


Overwriting job_search/job_search/spiders/spider.py


#### setting.py 수정

In [8]:
!head -n 22 job_search/job_search/settings.py | tail -n 2

# Obey robots.txt rules
ROBOTSTXT_OBEY = True


In [9]:
!sed -i 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' job_search/job_search/settings.py

sed: 1: "job_search/job_search/s ...": invalid command code j


In [10]:
!head -n 22 job_search/job_search/settings.py | tail -n 2

# Obey robots.txt rules
ROBOTSTXT_OBEY = True


#### Mongodb 모듈 파일 생성

In [11]:
%%writefile job_search/job_search/mongodb.py
import pymongo
import configparser

config = configparser.ConfigParser()
#configFilePath = r'E:ogin.ini'
config.read('login.ini')
user = config.get('ju', 'ID')
pwd = config.get('ju', 'PW')
ip = config.get('ju', 'IP')
account = f'mongodb://{user}:{pwd}@{ip}'
client=pymongo.MongoClient(account, 27017)
db=client.jobplanet
collection=db.review

Overwriting job_search/job_search/mongodb.py


#### pipeline 수정

In [14]:
%%writefile job_search/job_search/pipelines.py
from .mongodb import collection

class JobSearchPipeline(object):
    def process_item(self, item, spider):
        data={'company_name':item['company_name'], 'link':item['link'],'company_id':item['company_id'] , \
              'review_num':item['review_num'],'stats':item['stats'],'strength':item['strength'], \
              'title':item['title'], 'want':item['want'], 'weakness':item['weakness'],'person':item['person']}
        collection.insert(data)
        return item

Overwriting job_search/job_search/pipelines.py


In [15]:
!echo "ITEM_PIPELINES = {" >>job_search/job_search/settings.py

In [16]:
!echo " 'job_search.pipelines.JobSearchPipeline':300,">>job_search/job_search/settings.py

In [17]:
!echo "}">>job_search/job_search/settings.py

In [18]:
!tail -n 5 job_search/job_search/settings.py

#HTTPCACHE_IGNORE_HTTP_CODES = []
#HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'
ITEM_PIPELINES = {
 'job_search.pipelines.JobSearchPipeline':300,
}


In [19]:
!chmod +x run.sh

### argument 변경하며 실행

In [4]:
%%writefile run.sh
cd job_search
scrapy crawl JobSearch -o job_planet_ds_review.csv -a company_name='구글코리아'

Overwriting run.sh


### 실행
- run.sh 파일을 실행시키면 됌.
- 주피터 노트북에서도 매직커맨드와 함께 실행이 가능하지만, 저작권 문제를 피하기 위하여 실행하지 않겠음